In [ ]:
!pip install --upgrade pip
!pip install boto3 botocore

In [ ]:
import os
git_repo = f"https://<user>:<token>@huggingface.co/ds4sd/SmolDocling-256M-preview"
!git clone $git_repo

In [ ]:
import os

model_name = os.path.basename(git_repo)
model_name

In [ ]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)

#upload the model directory without git
def upload_directory_to_s3(local_directory, s3_prefix, remove_safetensors=True):
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            if ".git" in relative_path:
                print(f"skipping {relative_path}")
                continue
            # if remove_safetensors and ".safetensors" in relative_path:
            #     print(f"skipping {relative_path}")
            #     continue
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)


def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)

In [ ]:
list_objects("models")

In [ ]:
upload_directory_to_s3(model_name, f"models/{model_name}")

In [ ]:
list_objects("models")